# Mem0 Context Provider Example

This notebook demonstrates how to pair `Mem0Provider` with `AzureAIAgentClient` so an agent can remember user preferences across conversations.

Prerequisites:
- Run `az login` to authenticate with Azure.
- Set the `MEM0_API_KEY` environment variable (or pass the key explicitly to `Mem0Provider`).


In [ ]:
import asyncio
import uuid

from agent_framework.azure import AzureAIAgentClient
from agent_framework.mem0 import Mem0Provider
from azure.identity.aio import AzureCliCredential

print("Imports loaded.")

In [ ]:
def retrieve_company_report(company_code: str, detailed: bool) -> str:
    """Return a canned report for the CNTS company code.
    Raises ValueError if an unknown company code is provided."""
    if company_code != "CNTS":
        raise ValueError("Company code not found")
    if not detailed:
        return "CNTS is a company that specializes in technology."
    return (
        "CNTS is a company that specializes in technology. "
        "It had a revenue of $10 million in 2022. It has 100 employees."
    )


print("Helper tool registered.")

## Create the Mem0-enabled agent

The next cell builds an Azure AI agent and injects a Mem0 context provider so user preferences persist across threads.
It mirrors the `mem0_basic.py` sample shipped with the repository.


In [ ]:
async def main() -> None:
    """Run the Mem0 demonstration workflow."""
    print("=== Mem0 Context Provider Example ===")

    # Each Mem0 record must be associated with one of the supported identifiers.
    user_id = str(uuid.uuid4())
    print(f"Generated user id: {user_id}")

    # Authenticate with Azure and build the agent configured with Mem0.
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="FriendlyAssistant",
            instructions="You are a friendly assistant.",
            tools=retrieve_company_report,
            context_providers=Mem0Provider(user_id=user_id),
        ) as agent,
    ):
        # First request: agent lacks context so it should ask clarifying questions.
        query = "Please retrieve my company report"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result}\n")

        # Second request: provide preferences so they are stored via Mem0.
        query = "I always work with CNTS and I always want a detailed report format. Please remember and retrieve it."
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result}\n")

        print("\nRequest within a new thread:")
        thread = agent.get_new_thread()

        # Third request: the new thread starts empty, but Mem0 should replay preferences.
        query = "Please retrieve my company report"
        print(f"User: {query}")
        result = await agent.run(query, thread=thread)
        print(f"Agent: {result}\n")


print("Async workflow defined.")

In [ ]:
# Execute the coroutine in both notebook and script contexts.

try:
    loop = asyncio.get_running_loop()

except RuntimeError:
    asyncio.run(main())

else:
    if loop.is_running():
        await main()

    else:
        loop.run_until_complete(main())